# Assignment no 2
Name: Vishal Pattar \
Roll no: 33557 \
Class: TE AIML \
Subject: Artificial Intelligence for Cyber Security

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [25]:
df = pd.read_csv("./dataset_sdn.csv")
df.shape

(104345, 23)

In [26]:
df.sample(5)

,dt,switch,src,dst,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,...,pktrate,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label
35598,11319,3,10.0.0.6,10.0.0.5,27579,1489266,44,103000000,4.410300e+10,7,...,625,1,TCP,3,177052895,68545084,173,4057.0,4230.0,1
40718,25481,9,10.0.0.12,10.0.0.17,71556,4150248,227,980000000,2.280000e+11,5,...,295,1,TCP,2,7547119,6969828,0,0.0,0.0,1
26342,3489,4,10.0.0.3,10.0.0.5,87167,90828014,286,949000000,2.870000e+11,2,...,274,0,UDP,3,655718171,5657,2277,0.0,2277.0,1
29414,4119,5,10.0.0.8,10.0.0.5,133706,142530596,297,309000000,2.970000e+11,2,...,445,0,UDP,2,989339841,4238,3839,0.0,3839.0,0
71885,7717,3,10.0.0.3,10.0.0.16,14918,865244,79,894000000,7.989400e+10,11,...,98,1,TCP,1,9776,7022152,0,0.0,0.0,0


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104345 entries, 0 to 104344
Data columns (total 23 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   dt           104345 non-null  int64  
 1   switch       104345 non-null  int64  
 2   src          104345 non-null  object 
 3   dst          104345 non-null  object 
 4   pktcount     104345 non-null  int64  
 5   bytecount    104345 non-null  int64  
 6   dur          104345 non-null  int64  
 7   dur_nsec     104345 non-null  int64  
 8   tot_dur      104345 non-null  float64
 9   flows        104345 non-null  int64  
 10  packetins    104345 non-null  int64  
 11  pktperflow   104345 non-null  int64  
 12  byteperflow  104345 non-null  int64  
 13  pktrate      104345 non-null  int64  
 14  Pairflow     104345 non-null  int64  
 15  Protocol     104345 non-null  object 
 16  port_no      104345 non-null  int64  
 17  tx_bytes     104345 non-null  int64  
 18  rx_bytes     104345 non-

In [28]:
df.isnull().sum()

dt               0
switch           0
src              0
dst              0
pktcount         0
bytecount        0
dur              0
dur_nsec         0
tot_dur          0
flows            0
packetins        0
pktperflow       0
byteperflow      0
pktrate          0
Pairflow         0
Protocol         0
port_no          0
tx_bytes         0
rx_bytes         0
tx_kbps          0
rx_kbps        506
tot_kbps       506
label            0
dtype: int64

In [29]:
df = df.dropna()

In [30]:
df.describe()

,dt,switch,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,packetins,pktperflow,byteperflow,pktrate,Pairflow,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label
count,103839.000000,103839.000000,103839.000000,1.038390e+05,103839.000000,1.038390e+05,1.038390e+05,103839.000000,103839.000000,103839.000000,1.038390e+05,103839.000000,103839.000000,103839.000000,1.038390e+05,1.038390e+05,103839.000000,103839.000000,103839.000000,103839.000000
mean,17999.454165,4.214496,52781.703165,3.801729e+07,322.126118,4.613734e+08,3.225151e+11,5.666339,5196.261308,6365.870588,4.689472e+06,211.682248,0.603916,2.337580,9.370704e+07,9.373492e+07,1003.767322,1003.811420,2007.578742,0.390065
std,11962.227566,1.956320,52061.409178,4.874544e+07,283.926141,2.771020e+08,2.838104e+11,2.951013,5266.631950,7413.263909,7.562023e+06,247.138265,0.489085,1.082973,1.521679e+08,1.331642e+08,2428.363391,2054.887034,3144.437173,0.487767
min,2488.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,2.000000,4.000000,-130933.000000,-1.464426e+08,-4365.000000,0.000000,1.000000,2.527000e+03,8.560000e+02,0.000000,0.000000,0.000000,0.000000
25%,7128.000000,3.000000,801.000000,7.928200e+04,128.000000,2.330000e+08,1.280000e+11,3.000000,1943.000000,29.000000,2.842000e+03,0.000000,0.000000,1.000000,4.799000e+03,3.539000e+03,0.000000,0.000000,0.000000,0.000000
50%,11965.000000,4.000000,42557.000000,6.409692e+06,252.000000,4.180000e+08,2.520000e+11,5.000000,3024.000000,8304.000000,5.447360e+05,276.000000,1.000000,2.000000,4.552643e+06,1.401134e+07,0.000000,0.000000,4.000000,0.000000
75%,29982.000000,5.000000,94683.000000,7.597222e+07,413.000000,7.040000e+08,4.130000e+11,7.000000,7385.000000,10004.000000,9.727070e+06,333.000000,1.000000,3.000000,1.356509e+08,1.439286e+08,255.000000,557.000000,3838.000000,1.000000
max,42935.000000,10.000000,260006.000000,1.471280e+08,1881.000000,9.990000e+08,1.880000e+12,17.000000,25224.000000,19190.000000,1.495387e+07,639.000000,1.000000,5.000000,1.269982e+09,9.905962e+08,20580.000000,16577.000000,20580.000000,1.000000


In [31]:
numeric_df = df.select_dtypes(include=['int64', 'float64'])
object_df = df.select_dtypes(include=['object'])
numeric_cols = numeric_df.columns
object_cols = object_df.columns
print('Numeric Columns: ')
print(numeric_cols, '\n')
print('Object Columns: ')
print(object_cols, '\n')
print('Number of Numeric Features: ', len(numeric_cols))
print('Number of Object Features: ', len(object_cols))

Numeric Columns: 
Index(['dt', 'switch', 'pktcount', 'bytecount', 'dur', 'dur_nsec', 'tot_dur',
       'flows', 'packetins', 'pktperflow', 'byteperflow', 'pktrate',
       'Pairflow', 'port_no', 'tx_bytes', 'rx_bytes', 'tx_kbps', 'rx_kbps',
       'tot_kbps', 'label'],
      dtype='object') 

Object Columns: 
Index(['src', 'dst', 'Protocol'], dtype='object') 

Number of Numeric Features:  20
Number of Object Features:  3


In [32]:
label_encoder = LabelEncoder()
df['src'] = label_encoder.fit_transform(df['src'])
df['dst'] = label_encoder.fit_transform(df['dst'])
df['Protocol'] = label_encoder.fit_transform(df['Protocol'])

In [33]:
X = df.drop('label', axis=1)
y = df['label']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
def Evalute_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Accuracy Score: {:.2f}%".format(accuracy_score(y_test, y_pred)*100))
    print("Classification Report:\n{}".format(classification_report(y_test, y_pred))) 
    print("Confusion Matrix:\n{}".format(confusion_matrix(y_test, y_pred)))

In [36]:
Evalute_model(LogisticRegression())

Accuracy Score: 65.09%
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.81      0.74     12613
           1       0.58      0.40      0.47      8155

    accuracy                           0.65     20768
   macro avg       0.63      0.61      0.61     20768
weighted avg       0.64      0.65      0.63     20768

Confusion Matrix:
[[10240  2373]
 [ 4878  3277]]


e:\Programs\Python\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
Evalute_model(RandomForestClassifier())

Accuracy Score: 100.00%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12613
           1       1.00      1.00      1.00      8155

    accuracy                           1.00     20768
   macro avg       1.00      1.00      1.00     20768
weighted avg       1.00      1.00      1.00     20768

Confusion Matrix:
[[12613     0]
 [    0  8155]]


In [38]:
Evalute_model(DecisionTreeClassifier())

Accuracy Score: 100.00%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12613
           1       1.00      1.00      1.00      8155

    accuracy                           1.00     20768
   macro avg       1.00      1.00      1.00     20768
weighted avg       1.00      1.00      1.00     20768

Confusion Matrix:
[[12613     0]
 [    0  8155]]


In [39]:
Evalute_model(GaussianNB())

Accuracy Score: 62.35%
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.81      0.72     12613
           1       0.53      0.34      0.41      8155

    accuracy                           0.62     20768
   macro avg       0.59      0.57      0.57     20768
weighted avg       0.61      0.62      0.60     20768

Confusion Matrix:
[[10216  2397]
 [ 5423  2732]]
